# Entity Resolution Workshop

Entity Resolution is the task of disambiguating manifestations of real world entities through linking and grouping and is often an essential part of the data wrangling process. There are three primary tasks involved in entity resolution: deduplication, record linkage, and canonicalization; each of which serve to improve data quality by reducing irrelevant or repeated data, joining information from disparate records, and providing a single source of information to perform analytics upon. However, due to data quality issues (misspellings or incorrect data), schema variations in different sources, or simply different representations, entity resolution is not a straightforward process and most ER techniques utilize machine learning and other stochastic approaches.

In [27]:
## Imports

import os
import csv
import nltk
import json
import math
import random
import distance 
import re
import sys

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#reload(sys)  
#sys.setdefaultencoding('utf8')
## Important Paths
FIXTURES = os.path.join(os.getcwd(), "fixtures")
PRODUCTS = os.path.join(FIXTURES, "products")

## Module Constants
GOOGID   = 'http://www.google.com/base/feeds/snippets'

In [2]:
def load_data(name):
    """
    Create a generator to load data from the products data source.
    """
    with open(os.path.join(PRODUCTS, name), 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            yield row

def google_key(key):
    return os.path.join(GOOGID, key)

In [3]:
## Load Datasets into Memory
amazon  = list(load_data('amazon.csv'))
google  = list(load_data('google.csv'))
mapping = list(load_data('perfect_mapping.csv'))

## Report on contents of the dataset
for name, dataset in (('Amazon', amazon), ('Google Shopping', google)):
    print "{} dataset contains {} records".format(name, len(dataset))
    print "Record keys: {}\n".format(", ".join(dataset[0].keys()))

## Report on the contents of the mapping
print "There are {} matching records to link".format(len(mapping))

## Convert dataset to records indexed by their ID.
amazon  = dict((v['id'], v) for v in amazon)
google  = dict((v['id'], v) for v in google)

Amazon dataset contains 1363 records
Record keys: price, description, manufacturer, id, title

Google Shopping dataset contains 3226 records
Record keys: price, description, manufacturer, id, name

There are 1300 matching records to link


In [4]:
X = amazon['b0000c7fpt']
Y = google[google_key('17175991674191849246')]

## Show example Records
print json.dumps(X, indent=2)
print json.dumps(Y, indent=2)

{
  "price": "19.99", 
  "description": "reel deal casino shuffle master edition is packed with the hot casino games -- master them and become a high roller!", 
  "manufacturer": "phantom efx", 
  "id": "b0000c7fpt", 
  "title": "reel deal casino shuffle master edition"
}
{
  "price": "17.24", 
  "description": "reel deal casino shuffle master ed. is packed with an incredible 25 casino games featuring many of shuffle master's games that are sweeping the casino industry including let it ride crazy 4 poker triple shot and oklahoma 3-card. easily", 
  "manufacturer": "", 
  "id": "http://www.google.com/base/feeds/snippets/17175991674191849246", 
  "name": "phantom efx reel deal casino shuffle master edition"
}


## Similarity Scores

Links to information about distance metrics:

- [Implementing the Five Most Popular Similarity Measures in Python](http://dataconomy.com/implementing-the-five-most-popular-similarity-measures-in-python/)
- [Scikit-Learn Distance Metric](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html)
- [Python Distance Library](https://pypi.python.org/pypi/Distance/)

Numeric distances are fairly easy, but can be record specific (e.g. phone numbers can compare area codes, city codes, etc. to determine similarity). We will compare text similarity in this section:

In [5]:
# Typographic Distances

print distance.levenshtein("lenvestein", "levenshtein")
print distance.hamming("hamming", "hamning")

3
1


In [6]:
# Compare glyphs, syllables, or phonemes 
t1 = ("de", "ci", "si", "ve")
t2 = ("de", "ri", "si", "ve")
print distance.levenshtein(t1, t2)


1


In [7]:
# Sentence Comparison
sent1 = "The quick brown fox jumped over the lazy dogs."
sent2 = "The lazy foxes are jumping over the crazy Dog."

print distance.nlevenshtein(sent1.split(), sent2.split(), method=1)

0.666666666667


In [8]:
# Normalization
print distance.hamming("fat", "cat", normalized=True)
print distance.nlevenshtein("abc", "acd", method=1)  # shortest alignment
print distance.nlevenshtein("abc", "acd", method=2)  # longest alignment

0.333333333333
0.666666666667
0.5


In [10]:
# Set measures
print distance.sorensen("decide", "resize")
print distance.jaccard("decide", "decide")

0.555555555556
0.0


### Preprocessed Text Score

Use text preprocessing with NLTK to split long strings into parts, and normalize them using Wordnet.

In [11]:
def tokenize(sent):
    """
    When passed in a sentence, tokenizes and normalizes the string,
    returning a list of lemmata.
    """
    lemmatizer = nltk.WordNetLemmatizer() 
    for token in nltk.wordpunct_tokenize(sent):
        token = token.lower()
        yield lemmatizer.lemmatize(token)

def normalized_jaccard(*args):
    try:
        return distance.sorensen(*[tokenize(arg) for arg in args])
    except UnicodeDecodeError:
        return distance.sorensen(*[tokenize(unicode(arg, errors='ignore')) for arg in args])
    except ZeroDivisionError:  
        return 1

print normalized_jaccard(sent1, sent2)

0.333333333333


## Similarity Vectors

In [73]:



def similarity(prod1, prod2):
    """
    Returns a similarity vector of match scores:
    [name_score, description_score, manufacturer_score, price_score]
    """
    pair  = (prod1, prod2)
    names = [r.get('name', None) or r.get('title', None) for r in pair]
    descr = [r.get('description') for r in pair]
    manuf = [r.get('manufacturer') for r in pair]
    price = [float(r.get('price')) if r.get('price').isdigit() else float(re.findall('[\d\.]+',r.get('price'))[0]) for r in pair]
      
    return [
        1.0-normalized_jaccard(*names),
        1.0-normalized_jaccard(*descr),
        1.0-normalized_jaccard(*manuf),
        1.0-abs(price[0]-price[1])/max(price[0],price[1])
    ]

print similarity(X, Y)

[0.8571428571428571, 0.3728813559322034, 0.0, 0.8624312156078039]


## Weighted Pairwise Matching

In [79]:
%%time
import itertools
from joblib import Parallel, delayed
WEIGHTS   = (0.5, 0.2, 0.2, 0.1)
a = datetime.datetime.now()

azlist = amazon.values()[:20]
gglist = google.values()
joinlist = [azlist,gglist]
THRESHOLD = 0.4
def evaluate_similarity(azprod,googprod):
    vector = similarity(azprod, googprod)
    score  = sum(map(lambda v: v[0]*v[1], zip(WEIGHTS, vector)))

    return (score, azprod['id'], googprod['id'].split("/")[-1])


result_lst=Parallel(n_jobs=2)(delayed(evaluate_similarity)(each[0],each[1]) for each in itertools.product(*joinlist))

CPU times: user 720 ms, sys: 130 ms, total: 850 ms
Wall time: 46.3 s


In [80]:
result_lst

[(0.11284456327893755, 'b000ea9u2a', '13159207338580799968'),
 (0.16565697875725696, 'b000ea9u2a', '15932863893105585101'),
 (0.0022086861920527, 'b000ea9u2a', '18388451726220202579'),
 (0.10089783950266513, 'b000ea9u2a', '14837029540644805566'),
 (0.008688586073259164, 'b000ea9u2a', '11179585669816570240'),
 (0.025429341458379974, 'b000ea9u2a', '4649453787891480011'),
 (0.07950681335166895, 'b000ea9u2a', '18371419521576660155'),
 (0.054490311288911124, 'b000ea9u2a', '18424772291147437636'),
 (0.05631404780950644, 'b000ea9u2a', '10719377059258538369'),
 (0.16632496791690798, 'b000ea9u2a', '18434616791723899127'),
 (0.10329056738334791, 'b000ea9u2a', '16816523477506304009'),
 (0.04506228172138539, 'b000ea9u2a', '11041238021562109147'),
 (0.1810332517618342, 'b000ea9u2a', '8251934342990013076'),
 (0.0850981372671584, 'b000ea9u2a', '13561992178364532313'),
 (0.16581196581196583, 'b000ea9u2a', '14758105640999079833'),
 (0.07229524013630959, 'b000ea9u2a', '11830083407651005867'),
 (0.095549

In [81]:
matches = filter(lambda x: x[0]>THRESHOLD,result_lst)

for each in matches:
    print "{0:0.3f}: {1} {2}".format(each[0],each[1],each[2])

0.483: b000ea9u2a 18251424810598585817
0.512: b000ea9u2a 11826796220164615935
0.592: b000hkgj8k 18417354130985846422
0.433: b000fzvv3u 10282477376383378142
0.500: b000fzvv3u 12642359421626863460
0.654: b000fzvv3u 9675508034468866052
0.446: b000o39u8q 18204597739203681452
0.451: b000o39u8q 12766531527997249031
0.548: b000o39u8q 12958721621590299185
0.484: b000o39u8q 17939626833908573527
0.546: b000o39u8q 18295506345742178609
0.407: b0001ym04e 18420156205559716860
0.459: b0001ym04e 12514901799739690396
0.409: b000i84dum 18387170188665857199
0.411: b0009affew 10595032077010116055
0.518: b000e8jlda 14794198635011271295
0.415: b000e8jlda 9526578983773698753
0.411: b000hi1i9c 13499930330802525158
0.433: b0007lw230 17298610841039937504


In [82]:
a=amazon.get('b000o39u8q')
b=google.get(google_key('18204597739203681452'))
a
b
[name_score,desc_score,manu_score,price_score]=similarity(a,b)

print "name score: {0:0.3f} \ndescription score: {1:0.3f}\nmanufacturer score: {2:0.3f}\nprice score: {3:0.3f}".\
    format(name_score,desc_score,manu_score,price_score)
sum(map(lambda v: v[0]*v[1], zip(WEIGHTS, similarity(a,b))))

{'description': "elementary school advantage 2008 was specially developed to supplement classroom curriculum by including award-winning content that support state testing standards. in-depth lessons with sample problems and questions teach reinforce and track student progress in the core subjects. this award-winning content supports state testing standards and best of all it's easy to use & understand.",
 'id': 'b000o39u8q',
 'manufacturer': 'encore',
 'price': '39.99',
 'title': 'elementary advantage 2008'}

{'description': 'encore software 12361 : elementary school advantage 2008 was specially developed to supplement classroom curriculum by including award-winning content that support state testing standards. in-depth lessons with sample problems and questions teach ...',
 'id': 'http://www.google.com/base/feeds/snippets/18204597739203681452',
 'manufacturer': '',
 'name': 'encore software 12361 - encore elementary advantage 2008 - training/wbt - pc',
 'price': '33.97'}

name score: 0.429 
description score: 0.736
manufacturer score: 0.000
price score: 0.849


0.4463583876264633

In [42]:
?filter